In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')

In [ ]:
titanic_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv');

In [ ]:
titanic_data.head()

In [ ]:
# The NULL Values
sns.heatmap(titanic_data.isnull(),cmap='viridis')

In [ ]:
titanic_data['Survived'].value_counts()

# How many Females and Males were survived? Is Sex is an Imp Feature?

In [ ]:
titanic_data.groupby('Sex')['Survived'].value_counts()

In [ ]:
sns.countplot(x='Sex',hue='Survived',data=titanic_data)

Yes It IS!

# What is the survival rate if the person is a Male and Female?

In [ ]:
# Survival Rate= How many male were alived / Total number of males?
titanic_data.groupby('Sex')['Survived'].value_counts()[3]/titanic_data['Sex'].value_counts()[0]*100

In [ ]:
# Survival Rate= How many female were alived / Total number of Females?
titanic_data.groupby('Sex')['Survived'].value_counts()[0]/titanic_data['Sex'].value_counts()[1]*100

# How many Females and Males Travelling in different Pclass?

In [ ]:
titanic_data.groupby('Sex')['Pclass'].value_counts()

In [ ]:
sns.countplot(x='Sex',hue='Pclass',data=titanic_data,palette='cool')

# Which Passenger Class Contains more Fare in the basis of gender?


In [ ]:
titanic_data.groupby(['Pclass','Sex'])['Fare'].mean()

In [ ]:
titanic_data.groupby(['Pclass','Sex'])['Fare'].mean().plot(kind='bar')

In [ ]:
sns.barplot(x='Pclass',y='Fare',hue='Sex',data=titanic_data,palette='viridis')

# How many people were survived travelling in different Passenger Classes based on gender?

In [ ]:
titanic_data.groupby(['Pclass','Sex'])['Survived'].value_counts()

In [ ]:
plt.figure(figsize=(15,5))
titanic_data.groupby(['Pclass','Sex'])['Survived'].value_counts().sort_values(ascending=False).plot(kind='bar')

# How many People were travelling Alone? What's thier Survival Rate?

In [ ]:
alone=titanic_data[(titanic_data['Parch']==0) & (titanic_data['SibSp']==0)]

In [ ]:
alone['Survived'].value_counts(normalize=True)

So 537 People Were Travelling Alone, and Thier Survival Rate is around 30% which is lesser than the people travelling with the Siblings and Parents. So We have seen that the alone people has a low survival rate than the people who travelling with siblings and Parents

In [ ]:
print(titanic_data["SibSp"].value_counts())

print(titanic_data["Parch"].value_counts())

titanic_data['Family_size']= titanic_data['SibSp']+titanic_data['Parch']+1
test['Family_size']= test['SibSp']+test['Parch']+1

print(titanic_data["Family_size"].value_counts())

# Proportion of people survived for each class
print(titanic_data["Survived"].groupby(titanic_data["Family_size"]).mean().sort_values(ascending=False))

# Let's Now Handle the Missing Value

In [ ]:
titanic_data.isnull().sum()

Cabin: data missing > 70% . Cabin data can be ignored.

In [ ]:
titanic_data.drop(['Cabin'], axis=1, inplace=True)
test.drop(['Cabin'], axis=1, inplace=True)

In [ ]:
titanic_data.groupby(['Pclass','Sex'])['Age'].mean()

#### OK so we have find some Relations next let's do the same with the Name, means Extract the Titles from the name and then find mean of thise and check for that

In [ ]:
#creating column title
titanic_data['Title'] = titanic_data['Name'].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]
titanic_data['Title'].value_counts()

In [ ]:
test['Title'] = test['Name'].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]

### Here we have seen that the Common names/Most frequents are Mr, Miss,Mrs and Master so let's do accordingly

In [ ]:
Common_Title = ['Mr', 'Miss', 'Mrs', 'Master']
titanic_data['Title'].replace(['Ms', 'Mlle', 'Mme'], 'Miss', inplace=True)
titanic_data['Title'].replace(['Lady'], 'Mrs', inplace=True)
titanic_data['Title'].replace(['Sir', 'Rev'], 'Mr', inplace=True)
titanic_data['Title'][~titanic_data.Title.isin(Common_Title)] = 'Others'

In [ ]:
test['Title'].replace(['Ms', 'Mlle', 'Mme'], 'Miss', inplace=True)
test['Title'].replace(['Lady'], 'Mrs', inplace=True)
test['Title'].replace(['Sir', 'Rev'], 'Mr', inplace=True)
test['Title'][~test.Title.isin(Common_Title)] = 'Others'

In [ ]:
sns.boxplot(x='Title', y='Age', data=titanic_data)
plt.show()

In [ ]:
titanic_data.groupby(['Title'])['Age'].mean()

In [ ]:
titanic_data["Age"]=titanic_data.groupby('Title')['Age'].apply(lambda x:x.fillna(x.mean()))

In [ ]:
test["Age"]=test.groupby(['Pclass'])['Age'].apply(lambda x:x.fillna(x.mean()))

In [ ]:
# Replace The NaN value for Fare
test['Fare'].fillna(test['Fare'].median(),inplace=True)

In [ ]:
# Fill the Vallues with "S" in the place of NULL values of EMbarked column
titanic_data['Embarked'].fillna('S',inplace=True)

In [ ]:
print(titanic_data.isnull().sum())
print(test.isnull().sum())

# Fare

In [ ]:
# Fare
sns.distplot(titanic_data.Fare)
plt.show()

In [ ]:
titanic_data.boxplot('Fare')

In [ ]:
# Print 3 bins of Fare column
print(pd.cut(titanic_data['Fare'], 3).value_counts())

# Plot the histogram
sns.distplot(titanic_data["Fare"])
plt.show()

# Print binned Fares by surviving rate
print(titanic_data['Survived'].groupby(pd.cut(titanic_data['Fare'], 3)).mean())

In [ ]:
titanic_data.loc[titanic_data.Fare>512, 'Fare'] = 263
titanic_data.Fare.sort_values(ascending=False).head(5)

# Let's check the relationship between fare and family size

In [ ]:
titanic_data['FarePerPerson']= titanic_data['Fare']/titanic_data['Family_size']
titanic_data['FarePerPerson']

In [ ]:
test['FarePerPerson']= test['Fare']/test['Family_size']

# Now Let's See our Data

In [ ]:
titanic_data.head()

# One Hot Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()
titanic_data['Embarked']=label.fit_transform(titanic_data['Embarked'])
test['Embarked']=label.fit_transform(test['Embarked'])

In [ ]:
titanic_data['Sex']=label.fit_transform(titanic_data['Sex'])
test['Sex']=label.fit_transform(test['Sex'])

In [ ]:
titanic_data['Title']=label.fit_transform(titanic_data['Title'])
test['Title']=label.fit_transform(test['Title'])

In [ ]:
titanic_data.head()

# Feature Engineering

In [ ]:
# Drop Unnecessary Columns
titanic_data.drop(['Name','PassengerId','SibSp','Parch','Ticket'],axis=1,inplace=True)

In [ ]:
# Drop Unnecessary Columns
test.drop(['Name','SibSp','Parch','Ticket'],axis=1,inplace=True)

# Model Building

In [ ]:
X=titanic_data.drop('Survived',axis=1)
y=titanic_data['Survived']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=200, random_state=2)

# Feature Importance

In [ ]:
plt.rcParams["figure.figsize"] = (10,6)
model.fit(X_train,y_train)
importance = pd.DataFrame({'feature':X_train.columns, 'importance': np.round(model.feature_importances_,3)})
importance = importance.sort_values('importance', ascending=False).set_index('feature')
importance.plot(kind='bar', rot=0)
plt.show()

# Hyper Parameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
grid_param = {
 'n_estimators': [100, 200, 300],
 'criterion':['gini', 'entropy'],
 'min_samples_split': [2, 10, 20],
 'min_samples_leaf': [1, 5],
 'bootstrap': [True, False],
}
gd_sr = GridSearchCV(estimator=model,
 param_grid=grid_param,
 scoring='accuracy',
 cv=5,
 n_jobs=-1)
gd_sr.fit(X_train, y_train)
best_parameters = gd_sr.best_params_
print(best_parameters)

In [ ]:
model = RandomForestClassifier(n_estimators=200, bootstrap=False, criterion= 'gini', min_samples_leaf=5, min_samples_split=2, random_state=2)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,model.predict(X_test))*100

In [ ]:
from sklearn.model_selection import cross_val_score
Overall_accuracies = cross_val_score(estimator=model, X=X_train, y=y_train, cv=5)
print(Overall_accuracies)
Overall_accuracies.mean()*100

# So we have got 83% Accuracy Which is Pretty Good!

In [ ]:
test_data = test.drop("PassengerId", axis=1).copy()
prediction = model.predict(test_data)

In [ ]:
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": prediction
    })

submission.to_csv('submission.csv', index=False)

# Thanks A Lot.